In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## The problem ##

Let us consider the following problem, and how a Kalman filter can be used to solve it.  Imagine we are landing a rocket on a new planet.  For simplicity we approximate the ground as flat, the gravitational acceleration ($g$) as constant and the motion as one dimensional.  Adding complexity won't help us gain insight.

We are able to control the thrust of the rocket, and we obtain a series of measures of height, $h(t)$, at times $t_i$ spaced by $\Delta t=$const.  We don't know $g$ in advance.  How do we control the rocket to land softly?

## Kalman filter ##

In the standard notation our state vector, $\vec{x}$, advances from $k-1$ to $k$ via
$$
  \vec{x}_k = F\vec{x}_{k-1} + B\vec{u}_{k-1} + \epsilon_{k-1}
$$
where $F$ and $B$ are known matrices, $\vec{u}$ is my control variable and $\epsilon$ is the noise assumed normal:
$\epsilon_k\sim \mathcal{N}(0,Q_k)$.
In our case $\vec{x}=(h,v,g)$ and $\vec{u}=t$, the thrust, so
\begin{eqnarray}
  h_k &=& h_{k-1} + v_{k-1}\Delta t + \frac{1}{2}\left(t_{k-1} - g_{k-1}\right)\Delta t^2 \\
  v_k &=& v_{k-1} + (t_{k-1}-g_{k-1})\Delta t
\end{eqnarray}
with $g_k=g_{k-1}$.  This suggests
$$
  F = \left( \begin{array}{ccc}
        1 & \Delta t & -(1/2)\Delta t^2 \\
        0 &    1     & -\Delta t \\
        0 &    0     & 1
      \end{array}\right)
  \quad , \quad
  B = \left( \begin{array}{c}
        (1/2)\Delta t^2 \\ \Delta t \\ 0
      \end{array}\right)
$$

We then make an observation, $z$, with $z_{k} = H\vec{x}_k + \varepsilon_k$ and $\varepsilon_k\sim\mathcal{N}(0,R_k)$.

The Kalman steps are then:
1. $\hat{x} = Fx + Bu$
2. $P = FPF^T + Q$
3. $y = z - H\hat{x}$
4. $S = R + HPH^t$
5. $K = PH^TS^{-1}$
6. $x\to \hat{x} + Ky$
7. $P\to (1-KH)P$

In [ ]:
class Rocket:
    """A very simple class to hold and update the state."""
    def __init__(self,h):
        """Sets up the rocket at rest at height h."""
        self.h =  h
        self.v = 0.0
        self.g = 0.2  # Fixed but treated as unknown -- want to be small so dt=1 is a good timestep.
        self.t = 0.0
        #
    def advance(self,t):
        """Advance one click (Delta t=1), with thrust t."""
        self.t = t
        hnew   = self.h + self.v + 0.5*(self.t-self.g)
        vnew   =          self.v +     (self.t-self.g)
        self.h = hnew
        self.v = vnew
        #
    def read_thrust(self):
        return(self.t)
        #
    def read_state(self,sigma):
        """Returns the measured state.  Assume the error on v & g is huge and just return zeros."""
        err = sigma * np.random.normal(size=1)[0]
        return(np.array([self.h+err,0,0]))
        #
    def true_state(self):
        return(np.array([self.h,self.v,self.g]))
        #

In [ ]:
# We assume no buffeting from the atmosphere or fluctuations in the thrust vector.
# This means Q ~ identity times very small.
Q = 1e-3 * np.identity(3)
# Assume we measure all 3 components, so H=id(3).
# The measurement error is assumed to be 0.5m, with essentially no measure of v or g.
sigma = 0.5
R = np.diag([sigma,1e5,1e5])
#
P = np.zeros( (3,3) )
# Set up one Kalman step.
def KalmanStep(rocket,xold,Pold):
    F    = np.array([[1.0,1.0,-0.5],\
                     [0.0,1.0,-1.0],\
                     [0.0,0.0, 1.0]])
    B    = np.array( [0.5,1.0, 0.0])
    z    = rocket.read_state(sigma)
    xtry = np.dot(F,xold) + B*rocket.read_thrust()
    P    = np.dot(F,np.dot(Pold,F.T)) + Q
    K    = np.dot(P,np.linalg.inv(R+P))
    xnew = xtry + np.dot(K,z-xtry)
    Pnew = np.dot(np.identity(3)-K,P)
    return( (xnew,Pnew) )

In [ ]:
def pretty_print(i,thrust,xguess,xtrue):
    """Make a nicely formatted output for the routine below."""
    ss = "i={:3d}: t={:6.2f} ".format(i,thrust)
    ss+= " Kalm({:6.2f} {:6.2f} {:6.2f}) ".format(*xguess)
    ss+= " True({:6.2f} {:6.2f} {:6.2f}) ".format(*xtrue)
    print(ss)
    #

In [ ]:
rocket = Rocket(100.0)
xtrue  = rocket.true_state()
xest   = np.array([100.,0.,0.])
Parr   = np.zeros( (3,3) )
i=0
while xtrue[0]>0.05:
    xest,Parr = KalmanStep(rocket,xest,Parr)
    # Set the thrust to bring us to a soft-landing... this rule
    # isn't very clever, you can experiment with doing something
    # more sophisticated.  For example, we are not using the
    # fact that Kalman is giving us an error estimate (in P).
    tguess = np.max([0.,xest[2]-1*xest[0]-2*xest[1]])
    if i%5==0:
        pretty_print(i,tguess,xest,xtrue)
    rocket.advance(tguess)
    xtrue = rocket.true_state()
    i = i+1
pretty_print(i,tguess,xest,xtrue)

In [ ]:
# Now let's do this graphically instead.
rocket = Rocket(50.0)
xtrue  = rocket.true_state()
xest   = np.array([51.,0.,0.])
Parr   = np.zeros( (3,3) )
#
hTlst,vTlst = [xtrue[0]/10.],[-xtrue[1]]
hElst,vElst = [xest[0]/10.] ,[-xest[1]]
tlst = [0.0]
while xtrue[0]>0.05:
    xest,Parr = KalmanStep(rocket,xest,Parr)
    # Set the thrust to bring us to a soft-landing... this rule
    # isn't very clever, you can experiment with doing something
    # more sophisticated.  For example, we are not using the
    # fact that Kalman is giving us an error estimate (in P).
    tguess = np.max([0.,xest[2]-1*xest[0]-2*xest[1]])
    hTlst.append( xtrue[0]/10.)
    vTlst.append(-xtrue[1])
    hElst.append( xest[0]/10.)
    vElst.append(-xest[1])
    tlst.append(tguess)
    rocket.advance(tguess)
    xtrue = rocket.true_state()
tt = range(len(hTlst))
fig,ax = plt.subplots(1,1,figsize=(10,6))
ax.plot(tt,hTlst,'b--',label=r'hTrue/10')
ax.plot(tt,vTlst,'r--',label=r'vTrue')
ax.plot(tt,hElst,'b:' ,label=r'hEst/10')
ax.plot(tt,vElst,'r:' ,label=r'vEst')
ax.plot(tt,tlst ,'m-' ,label=r'Thrust')
ax.legend()
ax.set_xlabel('Time',fontsize=18)
ax.set_ylabel('Height and Speed',fontsize=18)